# Environment Setup

In [10]:
# generally used throughout the notebook
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

In [11]:
# Getting the documents

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'

docs_response = requests.get(docs_url)

dedicated_course_name = 'machine-learning-zoomcamp'
documents = list(filter(lambda doc: doc['course'] == dedicated_course_name, docs_response.json()))

len(documents)

375

In [12]:
# Getting the Gold standard Dataset

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

len(ground_truth)

1830

In [13]:
# propare the transformer
transformer = SentenceTransformer('multi-qa-distilbert-cos-v1')

# Q1. Getting the embeddings model

In [14]:
user_question = "I just discovered the course. Can I still join it?"

In [15]:
query_embedding = transformer.encode(user_question)

In [16]:
query_embedding[0]

0.07822263

In [17]:
query_embedding.shape[0]

768

# Q2. Creating the embeddings

In [18]:
documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

In [19]:
embeddings = []

for doc in tqdm(documents):
  embeddings.append(transformer.encode(f"{doc['question']} {doc['text']}"))

100%|██████████| 375/375 [04:03<00:00,  1.54it/s]


In [20]:
X = np.array(embeddings)

In [21]:
X.shape

(375, 768)

# Q3. Search

In [22]:
score = X.dot(query_embedding)

In [23]:
score.max()

0.65065753

# Implementing custom search engine

In [24]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

# Q4. Hit-rate for our search engine

In [25]:
search_engine = VectorSearchEngine(documents, X)

counter = 0
for item in tqdm(ground_truth):
  res = search_engine.search(transformer.encode(item['question']), num_results=5)
  res_ids = list(map(lambda ele: ele['id'], res))
  if item['document'] in res_ids:
    counter += 1

  0%|          | 0/1830 [00:00<?, ?it/s]

100%|██████████| 1830/1830 [05:20<00:00,  5.71it/s]


In [26]:
# calculating hit-rate
counter / len(ground_truth)

0.9398907103825137

# ElasticSearch setup

In [33]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

index_name = "ml-zoomcamp-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'ml-zoomcamp-questions'})

In [35]:
for idx, doc in tqdm(enumerate(documents)):
  es_client.index(index=index_name, document={**doc, 'question_text_vector': embeddings[idx]})

375it [00:05, 62.84it/s]


# Q5. Indexing with Elasticsearch

In [36]:
query = {
    "field": "question_text_vector",
    "query_vector": query_embedding,
    "k": 5,
    "num_candidates": 10000, 
}

In [47]:
es_client.search(index=index_name, knn=query, source=['id', 'text'])['hits']['hits'][0]

{'_index': 'ml-zoomcamp-questions',
 '_id': 'kPzMt5ABaBLTl4Dt71vN',
 '_score': 0.74546325,
 '_source': {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you can just use this link: http://mlzoomcamp.com/#syllabus',
  'id': '0a278fb2'}}

# Q6. Hit-rate for Elasticsearch

In [49]:
query =  {
  "field": "question_text_vector",
  "k": 5,
  "num_candidates": 10000, 
}

es_counter = 0
for item in tqdm(ground_truth):
  query['query_vector'] = transformer.encode(item['question'])
  res = es_client.search(index=index_name, knn=query, source=['id'])['hits']['hits']
  res_ids = list(map(lambda ele: ele['_source']['id'], res))
  if item['document'] in res_ids:
    es_counter += 1

100%|██████████| 1830/1830 [05:52<00:00,  5.19it/s]


In [50]:
es_counter / len(ground_truth)

0.9398907103825137

# Notes

In [27]:
from collections import defaultdict

repeatings = defaultdict(list)

for doc in documents:
  ln = len(df_ground_truth[df_ground_truth.document == doc['id']])

  repeatings[ln].append(doc['id'])

repeatings.keys()

dict_keys([5, 1])

In [28]:
len(repeatings[5])

365

In [29]:
len(repeatings[1])

10